# PHASE 2 - FORECASTING ABONNEMENTS FIBRE
## Comparaison de Modèles de Prévision Temporelle

**Objectif :** Comparer 5 modèles de prévision (Prophet, SARIMA, XGBoost, LSTM, Exponential Smoothing) pour prédire les abonnements quotidiens fibre sur 30-90 jours.

**Données :** Fibre Forecast Database (PostgreSQL)
- Période : 01/2024 → 12/2025 (730 jours)
- Cible : Nombre d'abonnements par jour
- Métrique succès : MAPE < 20%

## Section 1: Import Libraries & Configure Environment

In [ ]:
# Import libraries
import os
import sys
import warnings
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import time

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Machine Learning & Time Series
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import xgboost as xgb
from xgboost import XGBRegressor

# Time Series Models
from prophet import Prophet
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Database
from sqlalchemy import create_engine
import psycopg2

# Utils
from tqdm import tqdm
from dotenv import load_dotenv

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables
load_dotenv(dotenv_path='../.env')

print("✓ Toutes les librairies importées avec succès")

In [ ]:
# Metrics functions
def mae(y_true, y_pred):
    """Mean Absolute Error"""
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    """Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mape(y_true, y_pred):
    """Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    """Symmetric Mean Absolute Percentage Error"""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

def calculate_metrics(y_true, y_pred):
    """Calculate all metrics at once"""
    return {
        'MAE': mae(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE': mape(y_true, y_pred),
        'sMAPE': smape(y_true, y_pred)
    }

print("✓ Métriques définies")

## Section 2: Load Data from PostgreSQL

In [ ]:
# Database connection configuration
DB_USER = os.getenv('POSTGRES_USER', 'admin')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'SecurePassword123!')
DB_HOST = os.getenv('POSTGRES_HOST', 'localhost')
DB_PORT = os.getenv('POSTGRES_PORT', '5432')
DB_NAME = os.getenv('POSTGRES_DB', 'fibre_forecast_db')

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

print(f"🔗 Connexion à la base de données : {DB_HOST}:{DB_PORT}/{DB_NAME}")

# Load daily aggregated data
query_daily = """
    SELECT 
        DATE(f.created_at) as date,
        COUNT(*) as nb_abonnements
    FROM mart.fact_abonnements f
    GROUP BY DATE(f.created_at)
    ORDER BY date;
"""

df_daily = pd.read_sql(query_daily, engine)
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily = df_daily.sort_values('date').reset_index(drop=True)

# Load detailed data for feature engineering
query_detailed = """
    SELECT 
        DATE(f.created_at) as date,
        t.jour_semaine,
        t.mois,
        t.trimestre,
        t.est_weekend,
        t.est_ferie,
        g.governorate,
        o.categorie as offre_categorie,
        o.debit
    FROM mart.fact_abonnements f
    JOIN mart.dim_temps t ON f.date_id = t.date_id
    JOIN mart.dim_geographie g ON f.geo_id = g.geo_id
    JOIN mart.dim_offres o ON f.offre_id = o.offre_id
    ORDER BY date;
"""

df_detailed = pd.read_sql(query_detailed, engine)
df_detailed['date'] = pd.to_datetime(df_detailed['date'])

print(f"✓ Données chargées : {len(df_daily)} jours")
print(f"  Période : {df_daily['date'].min().date()} → {df_daily['date'].max().date()}")
print(f"  Total abonnements : {df_daily['nb_abonnements'].sum()}")
print(f"  Moyenne/jour : {df_daily['nb_abonnements'].mean():.2f}")

## Section 3: Exploratory Time Series Analysis

In [ ]:
# Visualize time series
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Full time series
axes[0].plot(df_daily['date'], df_daily['nb_abonnements'], linewidth=1.5, color='steelblue')
axes[0].fill_between(df_daily['date'], df_daily['nb_abonnements'], alpha=0.3, color='steelblue')
axes[0].set_title('Série Temporelle Complète - Abonnements Fibre Quotidiens', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Nombre d\'abonnements')
axes[0].grid(True, alpha=0.3)

# Statistics by day of week
df_daily['jour_semaine'] = df_daily['date'].dt.dayofweek
df_daily['sem_num'] = df_daily['date'].dt.isocalendar().week

daily_by_dow = df_daily.groupby('jour_semaine')['nb_abonnements'].mean()
dow_names = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
axes[1].bar(range(7), daily_by_dow.values, color='coral', alpha=0.7)
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(dow_names)
axes[1].set_title('Moyenne d\'abonnements par Jour de la Semaine', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Nombre d\'abonnements')
axes[1].grid(True, alpha=0.3, axis='y')

# Rolling mean
df_daily['rolling_7'] = df_daily['nb_abonnements'].rolling(window=7).mean()
df_daily['rolling_30'] = df_daily['nb_abonnements'].rolling(window=30).mean()
axes[2].plot(df_daily['date'], df_daily['nb_abonnements'], label='Quotidien', linewidth=0.8, alpha=0.5, color='gray')
axes[2].plot(df_daily['date'], df_daily['rolling_7'], label='Moyenne mobile 7j', linewidth=2, color='orange')
axes[2].plot(df_daily['date'], df_daily['rolling_30'], label='Moyenne mobile 30j', linewidth=2, color='red')
axes[2].set_title('Tendance avec Moyennes Mobiles', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Nombre d\'abonnements')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Autocorrelation analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(df_daily['nb_abonnements'].dropna(), lags=40, ax=axes[0])
plot_pacf(df_daily['nb_abonnements'].dropna(), lags=40, ax=axes[1])
axes[0].set_title('ACF - Autocorrelation Function')
axes[1].set_title('PACF - Partial Autocorrelation Function')
plt.tight_layout()
plt.show()

# Descriptive statistics
print("\n📊 Statistiques Descriptives:")
print(df_daily['nb_abonnements'].describe())
print(f"\nSkewness : {df_daily['nb_abonnements'].skew():.4f}")
print(f"Kurtosis : {df_daily['nb_abonnements'].kurtosis():.4f}")

# Check for missing dates
expected_days = (df_daily['date'].max() - df_daily['date'].min()).days + 1
missing_days = expected_days - len(df_daily)
print(f"\n⚠️  Jours manquants : {missing_days} / {expected_days}")

## Section 4: Feature Engineering for ML Models

In [ ]:
# Create feature engineering dataset
df_features = df_daily.copy()

# Temporal features
df_features['jour_mois'] = df_features['date'].dt.day
df_features['mois'] = df_features['date'].dt.month
df_features['trimestre'] = df_features['date'].dt.quarter
df_features['semaine_annee'] = df_features['date'].dt.isocalendar().week
df_features['est_weekend'] = df_features['jour_semaine'].isin([5, 6]).astype(int)

# Cyclical features
df_features['jour_annee'] = df_features['date'].dt.dayofyear
df_features['jour_annee_sin'] = np.sin(2 * np.pi * df_features['jour_annee'] / 365.25)
df_features['jour_annee_cos'] = np.cos(2 * np.pi * df_features['jour_annee'] / 365.25)

# Lag features (J-1, J-7, J-14, J-30)
for lag in [1, 7, 14, 30]:
    df_features[f'lag_{lag}'] = df_features['nb_abonnements'].shift(lag)

# Rolling mean features (7, 14, 30 days)
for window in [7, 14, 30]:
    df_features[f'rolling_mean_{window}'] = df_features['nb_abonnements'].rolling(window=window).mean()
    df_features[f'rolling_std_{window}'] = df_features['nb_abonnements'].rolling(window=window).std()

# Tunisian holidays
tunisian_holidays = pd.to_datetime([
    '2024-01-01', '2024-03-20', '2024-04-09', '2024-05-01',
    '2024-07-25', '2024-08-13', '2024-10-15',
    '2025-01-01', '2025-03-20', '2025-05-01',
    '2025-07-25', '2025-08-13', '2025-10-15'
])
df_features['est_ferie'] = df_features['date'].isin(tunisian_holidays).astype(int)

# Top 3 governorates and offers from detailed data
daily_gov = df_detailed.groupby(['date', 'governorate']).size().reset_index(name='count')
daily_offre = df_detailed.groupby(['date', 'offre_categorie']).size().reset_index(name='count')

top_govs = df_detailed['governorate'].value_counts().head(3).index.tolist()
top_offres = df_detailed['offre_categorie'].value_counts().head(3).index.tolist()

for gov in top_govs:
    gov_data = daily_gov[daily_gov['governorate'] == gov].groupby('date')['count'].sum()
    df_features = df_features.merge(
        gov_data.rename(f'gov_{gov}').to_frame().reset_index(),
        left_on='date', right_on='date', how='left'
    )
    df_features[f'gov_{gov}'] = df_features[f'gov_{gov}'].fillna(0)

for offre in top_offres:
    offre_data = daily_offre[daily_offre['offre_categorie'] == offre].groupby('date')['count'].sum()
    df_features = df_features.merge(
        offre_data.rename(f'offre_{offre}').to_frame().reset_index(),
        left_on='date', right_on='date', how='left'
    )
    df_features[f'offre_{offre}'] = df_features[f'offre_{offre}'].fillna(0)

# Drop rows with NaN from lags
df_features_clean = df_features.dropna()

print(f"✓ Features créées : {df_features_clean.shape[1] - 1} features")
print(f"✓ Lignes valides : {len(df_features_clean)} (après suppression NaN)")
print(f"\nFeatures disponibles :")
feature_cols = [col for col in df_features_clean.columns if col not in ['date', 'nb_abonnements']]
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")


## Section 5: Train/Test Split (Temporal)

In [ ]:
# Temporal train/test split (80/20)
train_size = int(len(df_features_clean) * 0.8)

df_train = df_features_clean.iloc[:train_size].copy()
df_test = df_features_clean.iloc[train_size:].copy()

# For time series models (Prophet, SARIMA, etc.)
train_ts = df_daily.iloc[:train_size].copy()
test_ts = df_daily.iloc[train_size:].copy()

print(f"📊 Train/Test Split (Temporal):")
print(f"  Train : {df_train['date'].min().date()} → {df_train['date'].max().date()} ({len(df_train)} jours)")
print(f"  Test  : {df_test['date'].min().date()} → {df_test['date'].max().date()} ({len(df_test)} jours)")
print(f"  Ratio : {len(df_train)/(len(df_train)+len(df_test))*100:.1f}% / {len(df_test)/(len(df_train)+len(df_test))*100:.1f}%")

# Prepare feature matrices for ML models
feature_cols = [col for col in df_features_clean.columns if col not in ['date', 'nb_abonnements']]

X_train = df_train[feature_cols].values
y_train = df_train['nb_abonnements'].values
X_test = df_test[feature_cols].values
y_test = df_test['nb_abonnements'].values

# For simple models (baseline)
y_train_full_ts = df_daily.iloc[:train_size]['nb_abonnements'].values
y_test_full_ts = df_daily.iloc[train_size:]['nb_abonnements'].values

print(f"\n✓ Features préparées : X_train {X_train.shape}, X_test {X_test.shape}")

# Visualize train/test split
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df_train['date'], df_train['nb_abonnements'], label='Train', linewidth=1.5, color='steelblue')
ax.plot(df_test['date'], df_test['nb_abonnements'], label='Test', linewidth=1.5, color='darkorange')
ax.axvline(df_train['date'].max(), color='red', linestyle='--', linewidth=2, label='Split')
ax.set_title('Train/Test Split Temporel', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Nombre d\'abonnements')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Section 6: Prophet Model Training & Evaluation

In [ ]:
print("🚀 PROPHET MODEL")

# Prepare data for Prophet (ds, y format)
df_prophet_train = pd.DataFrame({
    'ds': train_ts['date'].values,
    'y': train_ts['nb_abonnements'].values
})

df_prophet_test = pd.DataFrame({
    'ds': test_ts['date'].values,
    'y': test_ts['nb_abonnements'].values
})

# Tunisian holidays for Prophet
holidays = pd.DataFrame({
    'holiday': ['Nouvel An', 'Indépendance', 'Martyrs', 'Fête du Travail', 
                'République', 'Femme', 'Évacuation'] * 2,
    'ds': pd.to_datetime([
        '2024-01-01', '2024-03-20', '2024-04-09', '2024-05-01',
        '2024-07-25', '2024-08-13', '2024-10-15',
        '2025-01-01', '2025-03-20', '2025-05-01',
        '2025-07-25', '2025-08-13', '2025-10-15'
    ])
})

# Train Prophet with default parameters
start_time = time.time()
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    holidays=holidays,
    interval_width=0.95
)
prophet_model.fit(df_prophet_train)
prophet_time = time.time() - start_time

# Make predictions on test set
future_prophet_test = prophet_model.make_future_dataframe(periods=len(df_prophet_test))
future_prophet_test = future_prophet_test[future_prophet_test['ds'].isin(df_prophet_test['ds'])]
prophet_forecast = prophet_model.predict(future_prophet_test)

y_pred_prophet = prophet_forecast['yhat'].values[:len(df_prophet_test)]
prophet_metrics = calculate_metrics(df_prophet_test['y'].values, y_pred_prophet)
prophet_metrics['Time'] = prophet_time

print(f"  MAE: {prophet_metrics['MAE']:.4f}")
print(f"  RMSE: {prophet_metrics['RMSE']:.4f}")
print(f"  MAPE: {prophet_metrics['MAPE']:.2f}%")
print(f"  sMAPE: {prophet_metrics['sMAPE']:.2f}%")
print(f"  Temps d'entraînement: {prophet_time:.2f}s")

# Visualize Prophet predictions
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df_prophet_test['ds'], df_prophet_test['y'], label='Réel', linewidth=2, color='steelblue')
ax.plot(df_prophet_test['ds'], y_pred_prophet, label='Prédit (Prophet)', linewidth=2, color='darkorange', alpha=0.8)
ax.fill_between(df_prophet_test['ds'], 
                 prophet_forecast[prophet_forecast['ds'].isin(df_prophet_test['ds'])]['yhat_lower'],
                 prophet_forecast[prophet_forecast['ds'].isin(df_prophet_test['ds'])]['yhat_upper'],
                 alpha=0.2, color='orange')
ax.set_title('Prophet - Prédictions sur Test Set', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Nombre d\'abonnements')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Section 7: SARIMA/Auto-ARIMA Model Training & Evaluation

In [ ]:
print("🚀 SARIMA/AUTO-ARIMA MODEL")

# Auto-detect ARIMA parameters
start_time = time.time()
try:
    auto_model = auto_arima(
        y_train_full_ts,
        seasonal=True,
        m=7,  # Weekly seasonality
        stepwise=True,
        trace=False,
        max_p=5, max_q=5, max_d=2,
        max_P=2, max_Q=2, max_D=1,
        information_criterion='aic'
    )
    print(f"  Auto-ARIMA order: {auto_model.order}")
    print(f"  Seasonal order: {auto_model.seasonal_order}")
    
    # Fit on full training set and predict on test
    y_pred_sarima = auto_model.predict(n_periods=len(y_test_full_ts))
    sarima_time = time.time() - start_time
    
    sarima_metrics = calculate_metrics(y_test_full_ts, y_pred_sarima)
    sarima_metrics['Time'] = sarima_time
    
    print(f"  MAE: {sarima_metrics['MAE']:.4f}")
    print(f"  RMSE: {sarima_metrics['RMSE']:.4f}")
    print(f"  MAPE: {sarima_metrics['MAPE']:.2f}%")
    print(f"  sMAPE: {sarima_metrics['sMAPE']:.2f}%")
    print(f"  Temps d'entraînement: {sarima_time:.2f}s")
    
    # Visualize predictions
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.plot(test_ts['date'].values, y_test_full_ts, label='Réel', linewidth=2, color='steelblue')
    ax.plot(test_ts['date'].values, y_pred_sarima, label='Prédit (SARIMA)', linewidth=2, color='green', alpha=0.8)
    ax.set_title('SARIMA - Prédictions sur Test Set', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Nombre d\'abonnements')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"  ⚠️ Erreur SARIMA: {e}")
    sarima_metrics = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'sMAPE': np.nan, 'Time': 0}

## Section 8: XGBoost Model Training & Evaluation

In [ ]:
print("🚀 XGBOOST MODEL")

# Train XGBoost with default parameters
start_time = time.time()
xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0,
    tree_method='hist'
)
xgb_model.fit(X_train, y_train, verbose=False)
xgb_time = time.time() - start_time

# Predict on test set
y_pred_xgb = xgb_model.predict(X_test)
xgb_metrics = calculate_metrics(y_test, y_pred_xgb)
xgb_metrics['Time'] = xgb_time

print(f"  MAE: {xgb_metrics['MAE']:.4f}")
print(f"  RMSE: {xgb_metrics['RMSE']:.4f}")
print(f"  MAPE: {xgb_metrics['MAPE']:.2f}%")
print(f"  sMAPE: {xgb_metrics['sMAPE']:.2f}%")
print(f"  Temps d'entraînement: {xgb_time:.2f}s")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(range(len(feature_importance)), feature_importance['importance'].values, color='steelblue')
ax.set_yticks(range(len(feature_importance)))
ax.set_yticklabels(feature_importance['feature'].values)
ax.set_xlabel('Importance')
ax.set_title('XGBoost - Top 15 Features Importance', fontsize=12, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

# Visualize predictions
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df_test['date'].values, y_test, label='Réel', linewidth=2, color='steelblue')
ax.plot(df_test['date'].values, y_pred_xgb, label='Prédit (XGBoost)', linewidth=2, color='purple', alpha=0.8)
ax.set_title('XGBoost - Prédictions sur Test Set', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Nombre d\'abonnements')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Save XGBoost model for later
xgb_model_path = Path('../outputs/models/xgboost_default.pkl')
xgb_model_path.parent.mkdir(parents=True, exist_ok=True)
with open(xgb_model_path, 'wb') as f:
    pickle.dump(xgb_model, f)
print(f"\n✓ Modèle XGBoost sauvegardé: {xgb_model_path}")

## Section 9: LSTM Model Training & Evaluation

In [ ]:
print("🚀 LSTM MODEL")

# Prepare LSTM data with sequences (30-day windows)
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(y_train_full_ts.reshape(-1, 1)).flatten()

# Create sequences from training data
seq_length = 30
X_seq_train, y_seq_train = create_sequences(data_scaled, seq_length)

# Use test data scaled with training scaler
test_data_scaled = scaler.transform(y_test_full_ts.reshape(-1, 1)).flatten()
X_seq_test, y_seq_test = create_sequences(test_data_scaled, seq_length)

print(f"  Données LSTM : X_train {X_seq_train.shape}, X_test {X_seq_test.shape}")

# Build LSTM model
lstm_model = keras.Sequential([
    LSTM(64, return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train LSTM
start_time = time.time()
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = lstm_model.fit(
    X_seq_train.reshape(-1, seq_length, 1), y_seq_train,
    epochs=50,
    batch_size=16,
    verbose=0,
    callbacks=[early_stop]
)
lstm_time = time.time() - start_time

# Predict on test set
y_pred_lstm_scaled = lstm_model.predict(X_seq_test.reshape(-1, seq_length, 1), verbose=0)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm_scaled).flatten()

# Calculate metrics (only on available predictions)
min_len = min(len(y_test_full_ts), len(y_pred_lstm))
lstm_metrics = calculate_metrics(y_test_full_ts[:min_len], y_pred_lstm[:min_len])
lstm_metrics['Time'] = lstm_time

print(f"  MAE: {lstm_metrics['MAE']:.4f}")
print(f"  RMSE: {lstm_metrics['RMSE']:.4f}")
print(f"  MAPE: {lstm_metrics['MAPE']:.2f}%")
print(f"  sMAPE: {lstm_metrics['sMAPE']:.2f}%")
print(f"  Temps d'entraînement: {lstm_time:.2f}s")

# Visualize LSTM training history
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax.set_title('LSTM - Training History', fontsize=12, fontweight='bold')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Visualize LSTM predictions
fig, ax = plt.subplots(figsize=(14, 4))
# Adjust test dates to match predictions length
test_dates_adjusted = test_ts['date'].values[:len(y_pred_lstm)]
test_values_adjusted = y_test_full_ts[:len(y_pred_lstm)]
ax.plot(test_dates_adjusted, test_values_adjusted, label='Réel', linewidth=2, color='steelblue')
ax.plot(test_dates_adjusted, y_pred_lstm, label='Prédit (LSTM)', linewidth=2, color='red', alpha=0.8)
ax.set_title('LSTM - Prédictions sur Test Set', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Nombre d\'abonnements')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Save LSTM model
lstm_model_path = Path('../outputs/models/lstm_default.h5')
lstm_model_path.parent.mkdir(parents=True, exist_ok=True)
lstm_model.save(lstm_model_path)
print(f"\n✓ Modèle LSTM sauvegardé: {lstm_model_path}")

## Section 10: Exponential Smoothing Model Training & Evaluation

In [ ]:
print("🚀 EXPONENTIAL SMOOTHING MODEL")

# Exponential Smoothing Variants
models = {
    'SES': ExponentialSmoothing(y_train_full_ts, trend=None, seasonal=None),
    'Holt': ExponentialSmoothing(y_train_full_ts, trend='add', seasonal=None),
    'Holt-Winters': ExponentialSmoothing(y_train_full_ts, trend='add', seasonal='add', seasonal_periods=7)
}

best_es_model = None
best_es_metrics = None
best_es_name = None

for name, model in models.items():
    try:
        start_time = time.time()
        fitted = model.fit(optimized=True)
        y_pred_es = fitted.forecast(len(y_test_full_ts))
        es_time = time.time() - start_time
        
        metrics = calculate_metrics(y_test_full_ts, y_pred_es)
        metrics['Time'] = es_time
        
        print(f"\n  {name}:")
        print(f"    MAE: {metrics['MAE']:.4f}")
        print(f"    RMSE: {metrics['RMSE']:.4f}")
        print(f"    MAPE: {metrics['MAPE']:.2f}%")
        print(f"    sMAPE: {metrics['sMAPE']:.2f}%")
        print(f"    Temps: {es_time:.2f}s")
        
        # Keep best model
        if best_es_metrics is None or metrics['MAPE'] < best_es_metrics['MAPE']:
            best_es_metrics = metrics
            best_es_model = fitted
            best_es_name = name
    except Exception as e:
        print(f"  ⚠️ Erreur {name}: {e}")

if best_es_model:
    print(f"\n✓ Meilleur modèle Exponential Smoothing : {best_es_name}")

# Visualize best ES model
if best_es_model:
    y_pred_best_es = best_es_model.forecast(len(y_test_full_ts))
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.plot(test_ts['date'].values, y_test_full_ts, label='Réel', linewidth=2, color='steelblue')
    ax.plot(test_ts['date'].values, y_pred_best_es, label=f'Prédit ({best_es_name})', linewidth=2, color='green', alpha=0.8)
    ax.set_title(f'Exponential Smoothing ({best_es_name}) - Prédictions', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Nombre d\'abonnements')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Section 11: Hyperparameter Tuning for Top Models

In [ ]:
print("⚙️ HYPERPARAMETER TUNING")

# Prophet tuning (small grid for demo)
prophet_param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1],
    'seasonality_prior_scale': [0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

best_prophet_score = float('inf')
best_prophet_params = None

for cps in prophet_param_grid['changepoint_prior_scale']:
    for sps in prophet_param_grid['seasonality_prior_scale']:
        for mode in prophet_param_grid['seasonality_mode']:
            model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                holidays=holidays,
                changepoint_prior_scale=cps,
                seasonality_prior_scale=sps,
                seasonality_mode=mode
            )
            model.fit(df_prophet_train)
            future = model.make_future_dataframe(periods=len(df_prophet_test))
            future = future[future['ds'].isin(df_prophet_test['ds'])]
            forecast = model.predict(future)
            y_pred = forecast['yhat'].values[:len(df_prophet_test)]
            score = mape(df_prophet_test['y'].values, y_pred)
            if score < best_prophet_score:
                best_prophet_score = score
                best_prophet_params = (cps, sps, mode)

print(f"✅ Best Prophet params: {best_prophet_params}, MAPE: {best_prophet_score:.2f}%")

# XGBoost tuning (small grid)
xgb_param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1]
}

best_xgb_score = float('inf')
best_xgb_params = None

for n_estimators in xgb_param_grid['n_estimators']:
    for max_depth in xgb_param_grid['max_depth']:
        for lr in xgb_param_grid['learning_rate']:
            model = XGBRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=lr,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                verbosity=0,
                tree_method='hist'
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = mape(y_test, y_pred)
            if score < best_xgb_score:
                best_xgb_score = score
                best_xgb_params = (n_estimators, max_depth, lr)

print(f"✅ Best XGBoost params: {best_xgb_params}, MAPE: {best_xgb_score:.2f}%")

## Section 12: Model Comparison & Metrics Analysis

In [ ]:
# Aggregate metrics
comparison_data = {
    'Prophet': prophet_metrics,
    'SARIMA': sarima_metrics,
    'XGBoost': xgb_metrics,
    'LSTM': lstm_metrics,
    'Exp Smoothing': best_es_metrics
}

comparison_df = pd.DataFrame(comparison_data).T
comparison_df = comparison_df[['MAE', 'RMSE', 'MAPE', 'sMAPE', 'Time']]

print("📊 Model Comparison Table")
comparison_df.style.format('{:.3f}')

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
metrics = ['MAE', 'RMSE', 'MAPE', 'sMAPE']
for i, metric in enumerate(metrics):
    ax = axes[i//2, i%2]
    comparison_df[metric].plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title(f'Comparison by {metric}')
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identify best model
best_model_name = comparison_df['MAPE'].idxmin()
print(f"\n🏆 Best Model (MAPE): {best_model_name}")


## Section 13: Final Forecasts with Best Model

In [ ]:
print("📈 FINAL FORECASTS")

# Use best model for final forecast
if best_model_name == 'Prophet':
    final_model = prophet_model
    future = final_model.make_future_dataframe(periods=90)
    forecast = final_model.predict(future)
    forecast_final = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(90)
    
elif best_model_name == 'SARIMA':
    final_model = auto_model
    forecast_values = final_model.predict(n_periods=90)
    dates = pd.date_range(df_daily['date'].max() + pd.Timedelta(days=1), periods=90)
    forecast_final = pd.DataFrame({'ds': dates, 'yhat': forecast_values})

elif best_model_name == 'XGBoost':
    final_model = xgb_model
    last_date = df_features_clean['date'].max()
    future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=90)
    
    # Generate future features
    future_df = pd.DataFrame({'date': future_dates})
    future_df['jour_semaine'] = future_df['date'].dt.dayofweek
    future_df['jour_mois'] = future_df['date'].dt.day
    future_df['mois'] = future_df['date'].dt.month
    future_df['trimestre'] = future_df['date'].dt.quarter
    future_df['semaine_annee'] = future_df['date'].dt.isocalendar().week
    future_df['est_weekend'] = future_df['jour_semaine'].isin([5, 6]).astype(int)
    future_df['jour_annee'] = future_df['date'].dt.dayofyear
    future_df['jour_annee_sin'] = np.sin(2 * np.pi * future_df['jour_annee'] / 365.25)
    future_df['jour_annee_cos'] = np.cos(2 * np.pi * future_df['jour_annee'] / 365.25)
    
    # For lag/rolling features use last known values
    last_data = df_features_clean.tail(30)
    for lag in [1, 7, 14, 30]:
        future_df[f'lag_{lag}'] = last_data['nb_abonnements'].iloc[-lag]
    for window in [7, 14, 30]:
        future_df[f'rolling_mean_{window}'] = last_data['nb_abonnements'].rolling(window=window).mean().iloc[-1]
        future_df[f'rolling_std_{window}'] = last_data['nb_abonnements'].rolling(window=window).std().iloc[-1]
    
    # Holiday features
    future_df['est_ferie'] = future_df['date'].isin(tunisian_holidays).astype(int)
    
    # Placeholder for gov/offre (assume mean)
    for gov in top_govs:
        future_df[f'gov_{gov}'] = df_features_clean[f'gov_{gov}'].mean()
    for offre in top_offres:
        future_df[f'offre_{offre}'] = df_features_clean[f'offre_{offre}'].mean()

    # Predict
    X_future = future_df[feature_cols].values
    forecast_values = final_model.predict(X_future)
    forecast_final = pd.DataFrame({'ds': future_df['date'], 'yhat': forecast_values})

else:
    # Default fallback: Exponential Smoothing
    forecast_values = best_es_model.forecast(90)
    dates = pd.date_range(df_daily['date'].max() + pd.Timedelta(days=1), periods=90)
    forecast_final = pd.DataFrame({'ds': dates, 'yhat': forecast_values})

# Export forecasts
forecast_30 = forecast_final.head(30)
forecast_90 = forecast_final.head(90)

forecast_30.to_csv('../outputs/forecasts/forecast_30d.csv', index=False)
forecast_90.to_csv('../outputs/forecasts/forecast_90d.csv', index=False)

print("✓ Prévisions sauvegardées: forecast_30d.csv, forecast_90d.csv")

# Plot final forecast
plt.figure(figsize=(14, 5))
plt.plot(df_daily['date'], df_daily['nb_abonnements'], label='Historique', color='gray', linewidth=1.5)
plt.plot(forecast_final['ds'], forecast_final['yhat'], label='Prévisions', color='red', linewidth=2)
plt.fill_between(forecast_final['ds'], 
                 forecast_final.get('yhat_lower', forecast_final['yhat']),
                 forecast_final.get('yhat_upper', forecast_final['yhat']),
                 color='red', alpha=0.2)
plt.title('Prévisions 90 jours - Meilleur Modèle', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Nombre d\'abonnements')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Section 14: Visualization & Report Generation

In [ ]:
# Generate report in Markdown format
report_path = Path('../outputs/reports/model_comparison_report.md')
report_path.parent.mkdir(parents=True, exist_ok=True)

report_content = "# Model Comparison Report\n\n"
report_content += "## Summary\n\n"
report_content += f"Best Model: **{best_model_name}** (based on lowest MAPE)\n\n"
report_content += "## Metrics Table\n\n"
report_content += comparison_df.to_markdown() + "\n\n"
report_content += "## Insights\n\n"
report_content += "- Prophet: Handles seasonality and holiday effects well but may be slower.\n"
report_content += "- SARIMA: Strong baseline for seasonal data, interpretable.\n"
report_content += "- XGBoost: Captures complex feature interactions, often high accuracy.\n"
report_content += "- LSTM: Can model long dependencies but needs more data and tuning.\n"
report_content += "- Exp Smoothing: Simple and fast, best for stable trends.\n\n"
report_content += "## Recommendations\n\n"
report_content += "- Use the best model above for production forecasts.\n"
report_content += "- Re-train monthly as new data arrives.\n"
report_content += "- Monitor MAPE drift and retrain if MAPE > 25%.\n"

with open(report_path, 'w') as f:
    f.write(report_content)

print(f"✓ Rapport généré : {report_path}")
print("\n--- REPORT PREVIEW ---\n")
print(report_content[:1000])

In [ ]:
print("🚀 EXPONENTIAL SMOOTHING MODEL")

# Try different exponential smoothing variants
es_models = {}
es_metrics_list = []

start_time = time.time()

# Holt-Winters (with seasonal component)
try:
    hw_model = ExponentialSmoothing(
        y_train_full_ts,
        seasonal_periods=7,
        trend='add',
        seasonal='add',
        initialization_method='estimated'
    )
    hw_fit = hw_model.fit(optimized=True)
    y_pred_hw = hw_fit.forecast(steps=len(y_test_full_ts))
    es_metrics_hw = calculate_metrics(y_test_full_ts, y_pred_hw)
    es_metrics_hw['Method'] = 'Holt-Winters'
    es_metrics_list.append(es_metrics_hw)
    es_models['Holt-Winters'] = hw_fit
    print(f"  Holt-Winters MAPE: {es_metrics_hw['MAPE']:.2f}%")
except Exception as e:
    print(f"  ⚠️ Holt-Winters failed: {e}")

# Holt (linear trend, no seasonal)
try:
    from statsmodels.tsa.holtwinters import Holt
    holt_model = Holt(y_train_full_ts)
    holt_fit = holt_model.fit(optimized=True)
    y_pred_holt = holt_fit.forecast(steps=len(y_test_full_ts))
    es_metrics_holt = calculate_metrics(y_test_full_ts, y_pred_holt)
    es_metrics_holt['Method'] = 'Holt'
    es_metrics_list.append(es_metrics_holt)
    es_models['Holt'] = holt_fit
    print(f"  Holt MAPE: {es_metrics_holt['MAPE']:.2f}%")
except Exception as e:
    print(f"  ⚠️ Holt failed: {e}")

# Simple Exponential Smoothing
try:
    from statsmodels.tsa.holtwinters import SimpleExpSmoothing
    ses_model = SimpleExpSmoothing(y_train_full_ts)
    ses_fit = ses_model.fit(optimized=True)
    y_pred_ses = ses_fit.forecast(steps=len(y_test_full_ts))
    es_metrics_ses = calculate_metrics(y_test_full_ts, y_pred_ses)
    es_metrics_ses['Method'] = 'Simple ES'
    es_metrics_list.append(es_metrics_ses)
    es_models['Simple ES'] = ses_fit
    print(f"  Simple ES MAPE: {es_metrics_ses['MAPE']:.2f}%")
except Exception as e:
    print(f"  ⚠️ Simple ES failed: {e}")

es_time = time.time() - start_time

# Select best exponential smoothing model
best_es_idx = np.argmin([m['MAPE'] for m in es_metrics_list])
best_es_metrics = es_metrics_list[best_es_idx]
best_es_metrics['Time'] = es_time
best_es_method = es_metrics_list[best_es_idx]['Method']

print(f"\n  Best: {best_es_method}")
print(f"  MAE: {best_es_metrics['MAE']:.4f}")
print(f"  RMSE: {best_es_metrics['RMSE']:.4f}")
print(f"  MAPE: {best_es_metrics['MAPE']:.2f}%")
print(f"  sMAPE: {best_es_metrics['sMAPE']:.2f}%")
print(f"  Temps d'entraînement: {es_time:.2f}s")

# Visualize best exponential smoothing predictions
best_model = es_models[best_es_method]
y_pred_es_best = best_model.forecast(steps=len(y_test_full_ts))

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(test_ts['date'].values, y_test_full_ts, label='Réel', linewidth=2, color='steelblue')
ax.plot(test_ts['date'].values, y_pred_es_best, label=f'Prédit ({best_es_method})', linewidth=2, color='brown', alpha=0.8)
ax.set_title(f'Exponential Smoothing ({best_es_method}) - Prédictions sur Test Set', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Nombre d\'abonnements')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()